# MPRAnalyze Workflow for Expression Quantification using both V1 and V2

In [1]:
library(MPRAnalyze)
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


## Pool 1

In [2]:
# read in dataset
mpra_v1 <- read_tsv("../data/mpra_qtigc_pgl4v1_pool1.txt")
mpra_v2 <- read_tsv("../data/mpra_qtigc_pgl4v2_pool1.txt")

Rows: 50900 Columns: 16
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (3): snp, testcre, barcode
dbl (13): dna.r1, dna.r2, dna.r3, rna.r1, rna.r2, rna.r3, rna.r4, rna.r5, rn...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 50900 Columns: 16
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (3): snp, testcre, barcode
dbl (13): dna.r1, dna.r2, dna.r3, rna.r1, rna.r2, rna.r3, rna.r4, rna.r5, rn...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


## Formatting Data and Setting Up MPRAObject

In [3]:
num_enhancers_v1 <- dim(mpra_v1)[1]
num_enhancers_v2 <- dim(mpra_v2)[1]

In [4]:
num_enhancers_v1
num_enhancers_v2

[1] 50900

[1] 50900

In [5]:
# format DNA counts for v1
dna_counts_v1 <- mpra_v1 %>% select(-starts_with("rna"), -barcode, -snp) %>% 
    mutate(row=rep(c(1:50), times=num_enhancers_v1/50)) %>% 
    pivot_wider(names_from=row, values_from=starts_with("dna")) %>%
    rename_with(~paste0("V1.", .), -testcre)

In [6]:
# format DNA counts for v2
dna_counts_v2 <- mpra_v2 %>% select(-starts_with("rna"), -barcode, -snp) %>% 
    mutate(row=rep(c(1:50), times=num_enhancers_v2/50)) %>% 
    pivot_wider(names_from=row, values_from=starts_with("dna")) %>%
    rename_with(~paste0("V2.", .), -testcre)

In [7]:
# merge v1 and v2 DNA counts
dna_counts <- dna_counts_v1 %>% inner_join(dna_counts_v2, by = "testcre") %>%
    column_to_rownames("testcre") %>% as.matrix()

In [8]:
head(dna_counts)

,V1.dna.r1_1,V1.dna.r1_2,V1.dna.r1_3,V1.dna.r1_4,V1.dna.r1_5,V1.dna.r1_6,V1.dna.r1_7,V1.dna.r1_8,V1.dna.r1_9,V1.dna.r1_10,⋯,V2.dna.r3_41,V2.dna.r3_42,V2.dna.r3_43,V2.dna.r3_44,V2.dna.r3_45,V2.dna.r3_46,V2.dna.r3_47,V2.dna.r3_48,V2.dna.r3_49,V2.dna.r3_50
chr1_6147297_rs11583631_C_T_ref,0,1,0,0,0,0,0,0,0,7,⋯,0,0,0,5,0,0,12,0,2,0
chr1_6147297_rs11583631_C_T_alt,0,0,0,0,0,1,2,0,0,0,⋯,0,0,2,0,0,0,2,11,0,0
chr1_6147340_rs11584419_A_C_ref,0,0,0,0,1,0,0,0,0,0,⋯,2,0,1,0,5,0,0,0,0,0
chr1_6147340_rs11584419_A_C_alt,2,0,0,0,0,2,0,0,0,0,⋯,0,0,0,3,0,0,0,0,0,0
chr1_6157296_rs749435_T_C_ref,0,22,5,5,2,8,0,3,2,4,⋯,4,16,14,21,4,41,222,0,40,47
chr1_6157296_rs749435_T_C_alt,7,10,2,1,38,2,4,2,4,9,⋯,12,16,14,127,71,15,8,5,17,15


In [9]:
# format RNA counts
rna_counts_v1 <- mpra_v1 %>% select(-starts_with("dna"), -barcode, -snp) %>% 
    mutate(row=rep(c(1:50), times=num_enhancers_v1/50)) %>% 
    pivot_wider(names_from=row, values_from=starts_with("rna")) %>%
    rename_with(~paste0("V1.", .), -testcre)

In [10]:
# format RNA counts
rna_counts_v2 <- mpra_v2 %>% select(-starts_with("dna"), -barcode, -snp) %>% 
    mutate(row=rep(c(1:50), times=num_enhancers_v2/50)) %>% 
    pivot_wider(names_from=row, values_from=starts_with("rna")) %>%
    rename_with(~paste0("V2.", .), -testcre)

In [11]:
# merge v1 and v2 RNA counts
rna_counts <- rna_counts_v1 %>% inner_join(rna_counts_v2, by = "testcre") %>%
    column_to_rownames("testcre") %>% as.matrix()

In [12]:
head(rna_counts)

,V1.rna.r1_1,V1.rna.r1_2,V1.rna.r1_3,V1.rna.r1_4,V1.rna.r1_5,V1.rna.r1_6,V1.rna.r1_7,V1.rna.r1_8,V1.rna.r1_9,V1.rna.r1_10,⋯,V2.rna.r10_41,V2.rna.r10_42,V2.rna.r10_43,V2.rna.r10_44,V2.rna.r10_45,V2.rna.r10_46,V2.rna.r10_47,V2.rna.r10_48,V2.rna.r10_49,V2.rna.r10_50
chr1_6147297_rs11583631_C_T_ref,0,1,0,0,1,0,1,0,0,1,⋯,1,0,0,0,0,0,4,0,1,0
chr1_6147297_rs11583631_C_T_alt,0,0,0,0,0,0,1,0,0,0,⋯,0,0,5,0,0,0,1,6,0,0
chr1_6147340_rs11584419_A_C_ref,0,0,0,0,1,0,2,0,0,0,⋯,1,0,5,0,10,2,0,0,0,0
chr1_6147340_rs11584419_A_C_alt,2,0,0,0,0,1,1,0,4,0,⋯,0,0,0,3,0,0,0,0,0,2
chr1_6157296_rs749435_T_C_ref,4,30,3,22,9,7,1,0,10,3,⋯,6,52,24,18,13,75,251,3,44,84
chr1_6157296_rs749435_T_C_alt,7,29,2,6,70,12,12,1,3,18,⋯,17,6,9,142,94,34,4,2,14,30


In [13]:
# annotations for DNA data
dna_annot <- read.table("../data/dna_annot.txt", header=TRUE)

In [14]:
# annotations for RNA data
rna_annot <- read.table("../data/rna_annot_pool.txt", header=TRUE)

In [15]:
dna_annot <- dna_annot %>%
    mutate(version = as.factor(version)) %>%
    mutate(batch = as.factor(batch)) %>%
    mutate(barcode = as.factor(barcode))

In [16]:
rna_annot <- rna_annot %>%
    mutate(version = as.factor(version)) %>%
    mutate(batch = as.factor(batch)) %>%
    mutate(barcode = as.factor(barcode))

In [17]:
dim(dna_annot)
head(dna_annot)

[1] 300   3

,version,batch,barcode
,<fct>,<fct>,<fct>
V1:21:1,V1,21,1
V1:21:2,V1,21,2
V1:21:3,V1,21,3
V1:21:4,V1,21,4
V1:21:5,V1,21,5
V1:21:6,V1,21,6


In [18]:
dim(rna_annot)
head(rna_annot)

[1] 1000    3

,version,batch,barcode
,<fct>,<fct>,<fct>
V1:1:1,V1,1,1
V1:1:2,V1,1,2
V1:1:3,V1,1,3
V1:1:4,V1,1,4
V1:1:5,V1,1,5
V1:1:6,V1,1,6


In [19]:
# make MPRAObject
expr_obj <- MpraObject(dnaCounts = dna_counts, rnaCounts = rna_counts, 
                  dnaAnnot = dna_annot, rnaAnnot = rna_annot)

In [20]:
# perform library size normalization separately for DNA and RNA counts
expr_obj <- estimateDepthFactors(expr_obj, lib.factor = c("batch"),
                            which.lib = "dna", 
                            depth.estimator = "uq")
expr_obj <- estimateDepthFactors(expr_obj, lib.factor = c("batch"),
                            which.lib = "rna", 
                            depth.estimator = "uq")

## Quantification Analysis

In [21]:
# here only quantifying by batch
expr_obj <- analyzeQuantification(obj = expr_obj,
                                  dnaDesign = ~ version,
                                  rnaDesign = ~ 1)

Fitting model...


[--------------------------------------------------------------]   0% (5/1018)

[--------------------------------------------------------------]   1% (6/1018)

[--------------------------------------------------------------]   1% (7/1018)

[--------------------------------------------------------------]   1% (8/1018)

[>-------------------------------------------------------------]   1% (9/1018)

[>------------------------------------------------------------]   1% (10/1018)

[>------------------------------------------------------------]   1% (11/1018)

[>------------------------------------------------------------]   1% (12/1018)

[>------------------------------------------------------------]   1% (13/1018)

[>------------------------------------------------------------]   1% (14/1018)

[>------------------------------------------------------------]   1% (15/1018)

[>------------------------------------------------------------]   2% (16/1018)

[>-------------------

In [22]:
# extract alpha values (transcription rates) from the fitted model
expr_alpha <- getAlpha(expr_obj)

In [23]:
# split row names into enhancer information
names <- data.frame(do.call(rbind, strsplit(rownames(expr_alpha), "_")))
colnames(names) <- c("chrom", "pos", "snp", "allele", "mutation", "type")

In [24]:
# create new dataframe with snp, ref/alt, and alpha values
results <- data.frame('snp'= names$snp, 'type'=names$type, 'alpha'=expr_alpha$alpha)

In [25]:
# save MPRAnalyze results to text file
write.table(results, '../results/pool1_mpranalyze_results.txt')

## Pool 2

In [26]:
# read in dataset
mpra_v1_p2 <- read_tsv("../data/mpra_qtigc_pgl4v1_pool2.txt")
mpra_v2_p2 <- read_tsv("../data/mpra_qtigc_pgl4v2_pool2.txt")

Rows: 50900 Columns: 16
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (3): snp, testcre, barcode
dbl (13): dna.r1, dna.r2, dna.r3, rna.r1, rna.r2, rna.r3, rna.r4, rna.r5, rn...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 50900 Columns: 16
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (3): snp, testcre, barcode
dbl (13): dna.r1, dna.r2, dna.r3, rna.r1, rna.r2, rna.r3, rna.r4, rna.r5, rn...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


## Formatting Data and Setting Up MPRAObject

In [27]:
num_enhancers_v1_p2 <- dim(mpra_v1_p2)[1]
num_enhancers_v2_p2 <- dim(mpra_v2_p2)[1]

In [28]:
num_enhancers_v1_p2
num_enhancers_v2_p2

[1] 50900

[1] 50900

In [29]:
# format DNA counts for v1
dna_counts_v1_p2 <- mpra_v1_p2 %>% select(-starts_with("rna"), -barcode, -snp) %>% 
    mutate(row=rep(c(1:50), times=num_enhancers_v1_p2/50)) %>% 
    pivot_wider(names_from=row, values_from=starts_with("dna")) %>%
    rename_with(~paste0("V1.", .), -testcre)

In [30]:
# format DNA counts for v2
dna_counts_v2_p2 <- mpra_v2_p2 %>% select(-starts_with("rna"), -barcode, -snp) %>% 
    mutate(row=rep(c(1:50), times=num_enhancers_v2_p2/50)) %>% 
    pivot_wider(names_from=row, values_from=starts_with("dna")) %>%
    rename_with(~paste0("V2.", .), -testcre)

In [31]:
# merge v1 and v2 DNA counts
dna_counts_p2 <- dna_counts_v1_p2 %>% inner_join(dna_counts_v2_p2, by = "testcre") %>%
    column_to_rownames("testcre") %>% as.matrix()

In [32]:
head(dna_counts_p2)

,V1.dna.r1_1,V1.dna.r1_2,V1.dna.r1_3,V1.dna.r1_4,V1.dna.r1_5,V1.dna.r1_6,V1.dna.r1_7,V1.dna.r1_8,V1.dna.r1_9,V1.dna.r1_10,⋯,V2.dna.r3_41,V2.dna.r3_42,V2.dna.r3_43,V2.dna.r3_44,V2.dna.r3_45,V2.dna.r3_46,V2.dna.r3_47,V2.dna.r3_48,V2.dna.r3_49,V2.dna.r3_50
chr2_201172085_rs67190025_C_T_ref,0,3,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,7,0,4,9
chr2_201172085_rs67190025_C_T_alt,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,1,0,0,9,0,0
chr2_201172627_rs7580924_G_C_ref,0,0,0,6,1,0,6,1,5,0,⋯,2,4,4,0,0,24,2,9,10,0
chr2_201172627_rs7580924_G_C_alt,0,6,0,0,0,0,2,0,0,0,⋯,17,0,10,0,8,13,4,1,1,0
chr2_201185918_rs13028959_C_A_ref,47,64,34,52,21,36,2,13,73,32,⋯,30,53,50,135,20,170,299,85,661,34
chr2_201185918_rs13028959_C_A_alt,21,39,29,8,14,25,24,27,45,15,⋯,440,18,86,41,226,106,270,98,59,191


In [33]:
# format RNA counts
rna_counts_v1_p2 <- mpra_v1_p2 %>% select(-starts_with("dna"), -barcode, -snp) %>% 
    mutate(row=rep(c(1:50), times=num_enhancers_v1_p2/50)) %>% 
    pivot_wider(names_from=row, values_from=starts_with("rna")) %>%
    rename_with(~paste0("V1.", .), -testcre)

In [34]:
# format RNA counts
rna_counts_v2_p2 <- mpra_v2_p2 %>% select(-starts_with("dna"), -barcode, -snp) %>% 
    mutate(row=rep(c(1:50), times=num_enhancers_v2_p2/50)) %>% 
    pivot_wider(names_from=row, values_from=starts_with("rna")) %>%
    rename_with(~paste0("V2.", .), -testcre)

In [35]:
# remove low quality replicate (v1:r2 and v2:r7)
rna_counts_v1_p2 <- rna_counts_v1_p2 %>%
    select(-c(starts_with("V1.rna.r2")))

rna_counts_v2_p2 <- rna_counts_v2_p2 %>%
    select(-c(starts_with("V2.rna.r7")))

In [36]:
# merge v1 and v2 RNA counts
rna_counts_p2 <- rna_counts_v1_p2 %>% inner_join(rna_counts_v2_p2, by = "testcre") %>%
    column_to_rownames("testcre") %>% as.matrix()

In [37]:
dim(rna_counts_p2)

[1] 1018  900

In [38]:
# adjust rna_annot accordingly
rna_annot_p2 <- rna_annot %>% 
    filter(!(version=="V1" & batch==2)) %>%
    filter(!(version=="V2" & batch==17))

In [39]:
# make MPRAObject
# annotation is identical
expr_obj_p2 <- MpraObject(dnaCounts = dna_counts_p2, rnaCounts = rna_counts_p2,
                  dnaAnnot = dna_annot, rnaAnnot = rna_annot_p2)

In [40]:
# perform library size normalization separately for DNA and RNA counts
expr_obj_p2 <- estimateDepthFactors(expr_obj_p2, lib.factor = c("batch"),
                            which.lib = "dna", 
                            depth.estimator = "uq")
expr_obj_p2 <- estimateDepthFactors(expr_obj_p2, lib.factor = c("batch"),
                            which.lib = "rna", 
                            depth.estimator = "uq")

## Quantification Analysis

In [41]:
# here only quantifying by batch
expr_obj_p2 <- analyzeQuantification(obj = expr_obj_p2,
                                  dnaDesign = ~ version,
                                  rnaDesign = ~ 1)

Fitting model...


[--------------------------------------------------------------]   0% (5/1018)

[--------------------------------------------------------------]   1% (6/1018)

[--------------------------------------------------------------]   1% (7/1018)

[--------------------------------------------------------------]   1% (8/1018)

[>-------------------------------------------------------------]   1% (9/1018)

[>------------------------------------------------------------]   1% (10/1018)

[>------------------------------------------------------------]   1% (11/1018)

[>------------------------------------------------------------]   1% (12/1018)

[>------------------------------------------------------------]   1% (13/1018)

[>------------------------------------------------------------]   1% (14/1018)

[>------------------------------------------------------------]   1% (15/1018)

[>------------------------------------------------------------]   2% (16/1018)

[>-------------------

In [42]:
# extract alpha values (transcription rates) from the fitted model
expr_alpha_p2 <- getAlpha(expr_obj_p2)

In [43]:
# split row names into enhancer information
names_p2 <- data.frame(do.call(rbind, strsplit(rownames(expr_alpha_p2), "_")))
colnames(names_p2) <- c("chrom", "pos", "snp", "allele", "mutation", "type")

In [44]:
# create new dataframe with snp, ref/alt, and alpha values
results_p2 <- data.frame('snp'= names_p2$snp, 'type'=names_p2$type, 'alpha'=expr_alpha_p2$alpha)

In [45]:
# save MPRAnalyze results to text file
write.table(results_p2, '../results/pool2_mpranalyze_results.txt')